In [ ]:
from helpers.db.db_setup_methods import *
from helpers.db.db_query_methods import *

from helpers.db.db_helper_methods import *

from helpers.es.es_setup_methods import *
from helpers.es.es_helper_methods import *

records_size = 200_000

db_name_no_index = "db_no_index"
db_name_with_index = "db_with_index"

init_db(records_max=records_size, db_name=db_name_no_index)
init_db(records_max=records_size, db_name=db_name_with_index)

es_standard_index = "standard_index"

init_index(index_name=es_standard_index, documents_records=records_size)

First the db benchmark

In [ ]:
db_query = """
SELECT * FROM products WHERE name = 'Pro Charger Tablet Brush Go 360'
"""

execute_query(query=db_query, database=db_name_no_index, print_as_df=False, show_metrics=True)

Now creating a index to check the best possible speed

In [ ]:
db_query = """
CREATE INDEX idx_products_name ON products (name)
"""

execute_query(query=db_query, database=db_name_with_index, print_as_df=False, show_metrics=False)

In [ ]:
db_query = """
SELECT * FROM products WHERE name = 'Pro Charger Tablet Brush Go 360'
"""

execute_query(query=db_query, database=db_name_with_index, print_as_df=False, show_metrics=True)

Now testing out ES

In [ ]:
user_input = "Pro Charger Tablet Brush Go 360"

query = {
    "query": {
        "match": {
            "name": user_input
        }
    }
}

search_text(query, es_standard_index)

This seems to acutally be the DB that is faster than ES. I would have thought they would either be around the same speed or that es would be quicker.

Now lets try to only include the first half of the search to see if ES now performs faster. I would expect so. Since the DB cant use the index for a exact match

In [ ]:
db_query = """
SELECT * FROM products WHERE name like 'Pro Charger Tablet%%';
"""

execute_query(query=db_query, database=db_name_with_index, print_as_df=False, show_metrics=True)

In [ ]:
user_input = "Pro Charger Tablet"

query = {
    "query": {
        "match_phrase": {
            "name": user_input
        }
    }
}

search_text(query, es_standard_index)

So This yielded about the same result. i did not expect that.

Next lets force the DB to not be able to use the index by doing a wildcard on both sides

In [ ]:
db_query = """
SELECT * FROM products WHERE name like '%%Charger Tablet%%';
"""

execute_query(query=db_query, database=db_name_with_index, print_as_df=False, show_metrics=True)

In [ ]:
user_input = "Charger Tablet"

query = {
    "query": {
        "match_phrase": {
            "name": user_input
        }
    }
}

search_text(query, es_standard_index)

Now the difference is clear. Es has mentained the exact same speed through out whereas the DB gets slower and slower depnding on either the selectivity or if it has to do a full table scan